https://newtechaudit.ru/sozdanie-novyh-priznakov-dlya-povysheniya-kachestva-mashinnogo-obucheniya/

https://newtechaudit.ru/razrabotka-priznakov/#:~:text=%D0%9F%D0%BE%D0%BB%D0%B8%D0%BD%D0%BE%D0%BC%D0%B8%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5%20%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA%D0%B8,-%D0%9E%D0%B4%D0%BD%D0%B8%D0%BC%20%D0%B8%D0%B7%20%D0%BF%D1%80%D0%BE%D1%81%D1%82%D1%8B%D1%85&text=%D0%AD%D1%82%D0%B8%20%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA%D0%B8%2C%20%D0%BF%D1%80%D0%B5%D0%B4%D1%81%D1%82%D0%B0%D0%B2%D0%BB%D1%8F%D1%8E%D1%89%D0%B8%D0%B5%20%D1%81%D0%BE%D0%B1%D0%BE%D0%B9%20%D0%BA%D0%BE%D0%BC%D0%B1%D0%B8%D0%BD%D0%B0%D1%86%D0%B8%D1%8E,%D0%BE%D0%BD%D0%B8%20%D1%84%D0%B8%D0%BA%D1%81%D0%B8%D1%80%D1%83%D1%8E%D1%82%20%D0%B2%D0%B7%D0%B0%D0%B8%D0%BC%D0%BE%D0%B4%D0%B5%D0%B9%D1%81%D1%82%D0%B2%D0%B8%D1%8F%20%D0%BC%D0%B5%D0%B6%D0%B4%D1%83%20%D0%BF%D0%B5%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D0%BC%D0%B8.

# Трунов Никита,
# Билет 0

## Описание датасета

Датасет представляет собой набор данных об оттоке клиентов компании Orange Telecom. Датасет содержит очищенные данных об активности клиентов, а также информацию о том, отменил ли клиент подписку. Решаемая задача - задача классификации.

## Импорты

In [1]:
import warnings
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import (
    VarianceThreshold,
    SelectFromModel,
    RFE,
    SequentialFeatureSelector,
    SelectKBest,
    chi2,
    f_classif
)
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import (
    make_scorer,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
warnings.filterwarnings("ignore")

## Загрузка датасета

In [3]:
df = pd.read_csv('churn-bigml-80.csv').append(
    pd.read_csv('churn-bigml-20.csv')
)
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


## Анализ и обработка данных

### Общие характеристики датасета

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3333 entries, 0 to 666
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   3333 non-null   object 
 1   Account length          3333 non-null   int64  
 2   Area code               3333 non-null   int64  
 3   International plan      3333 non-null   object 
 4   Voice mail plan         3333 non-null   object 
 5   Number vmail messages   3333 non-null   int64  
 6   Total day minutes       3333 non-null   float64
 7   Total day calls         3333 non-null   int64  
 8   Total day charge        3333 non-null   float64
 9   Total eve minutes       3333 non-null   float64
 10  Total eve calls         3333 non-null   int64  
 11  Total eve charge        3333 non-null   float64
 12  Total night minutes     3333 non-null   float64
 13  Total night calls       3333 non-null   int64  
 14  Total night charge      3333 non-null   f

In [5]:
df.shape

(3333, 20)

Исходный датасет содержит 3333 точек данных и 20 переменных.

### Обработка пропусков

In [6]:
df.isna().sum()

State                     0
Account length            0
Area code                 0
International plan        0
Voice mail plan           0
Number vmail messages     0
Total day minutes         0
Total day calls           0
Total day charge          0
Total eve minutes         0
Total eve calls           0
Total eve charge          0
Total night minutes       0
Total night calls         0
Total night charge        0
Total intl minutes        0
Total intl calls          0
Total intl charge         0
Customer service calls    0
Churn                     0
dtype: int64

Отсутствующие значения в выборке не обнаружены.

### Выделение признаков и целевой переменной

In [7]:
X = df.drop(columns=['Churn'])
y = df['Churn']

### Кодирование категориальных признаков

In [8]:
X = pd.get_dummies(X)

Используем способ "One Hot Encoding" для кодирования категориальных признаков.

In [9]:
X.shape

(3333, 71)

В результате получаем 71 признак.

### Разделение выборки на обучающую и тестовую

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

Разделяем выборку на обучающую и тестовую в соотношении 80% и 20%.

## Обучение модели

In [11]:
clf = KNeighborsClassifier()

В качестве классификатора будем использовать метод k ближайших соседей.

In [12]:
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, pos_label=True),
    'recall': make_scorer(recall_score, pos_label=True),
    'f1': make_scorer(f1_score, pos_label=True)
}

В качестве метрик качества будем использовать accuracy, precision, recall и f1.

In [13]:
def get_cv_metrics(estimator, X, y, scoring):
    return pd.Series({
        metric: values.mean()
        for metric, values in cross_validate(
            estimator, X, y, scoring=scoring).items()
    }, name=f'{type(estimator).__name__}')

Для определения эффективности модели будем считать усредненные метрики в процессе проведения кросс-валидации. Она необходима, чтобы понимать, насколько полученные метрики валидны и подобрать на их основании наиболее эффективную модель.

In [14]:
get_cv_metrics(clf, X_train, y_train, scoring)

fit_time          0.005200
score_time        0.093799
test_accuracy     0.869846
test_precision    0.672392
test_recall       0.244430
test_f1           0.356811
Name: KNeighborsClassifier, dtype: float64

## Выбор признаков

### Удаление константных признаков

In [15]:
vt = VarianceThreshold().fit(X_train)
X_train = vt.transform(X_train)
X_test = vt.transform(X_test)

Удалим признаки, дисперсия которых равная 0.

In [16]:
X_train.shape

(2666, 71)

### Одномерный отбор

#### По тесту ХИ-квадрат

In [17]:
chi2_skb = SelectKBest(chi2, k=6).fit(X_train, y_train)

Отбор признаков с наилучшей статистикой ХИ-квадрат. 

In [18]:
get_cv_metrics(clf, chi2_skb.transform(X_train), y_train, scoring)

fit_time          0.004000
score_time        0.021200
test_accuracy     0.884844
test_precision    0.742840
test_recall       0.345000
test_f1           0.470761
Name: KNeighborsClassifier, dtype: float64

In [19]:
chi2_skb.transform(X_train).shape

(2666, 6)

#### По F-тесту

In [20]:
f_skb = SelectKBest(f_classif, k=10).fit(X_train, y_train)

Отбор признаков с наилучшей F-статистикой. 

In [21]:
get_cv_metrics(clf, f_skb.transform(X_train), y_train, scoring)

fit_time          0.005205
score_time        0.020600
test_accuracy     0.885595
test_precision    0.748809
test_recall       0.347532
test_f1           0.474195
Name: KNeighborsClassifier, dtype: float64

In [22]:
f_skb.transform(X_train).shape

(2666, 10)

### Выбор на основе моделей

#### Выбор на основе линейной модели

In [23]:
lg_sfm = SelectFromModel(LogisticRegression()).fit(X_train, y_train)

При обучении на линейных моделях веса можно интерпретировать как важность признака.

In [24]:
get_cv_metrics(clf, lg_sfm.transform(X_train), y_train, scoring)

fit_time          0.003399
score_time        0.022400
test_accuracy     0.867591
test_precision    0.645881
test_recall       0.264715
test_f1           0.370424
Name: KNeighborsClassifier, dtype: float64

In [25]:
lg_sfm.transform(X_train).shape

(2666, 5)

#### Выбор на основе модели случайного леса

In [26]:
rfc_sfm = SelectFromModel(RandomForestClassifier()).fit(X_train, y_train)

При обучении на моделях случайного леса то, насколько признак снижает энтропию, можно интерпретировать как важность признака.

In [27]:
get_cv_metrics(clf, rfc_sfm.transform(X_train), y_train, scoring)

fit_time          0.000400
score_time        0.076000
test_accuracy     0.874345
test_precision    0.697740
test_recall       0.277120
test_f1           0.394938
Name: KNeighborsClassifier, dtype: float64

In [28]:
rfc_sfm.transform(X_train).shape

(2666, 17)

### Выбор на основе рекурсивной оценки

#### Рекурсивная оценка на основе линейной модели

In [29]:
lf_rfe = RFE(LogisticRegression(), n_features_to_select=8).fit(X_train, y_train)

Попробуем рекурсивно перебрать все признаки и на основании весов, полученных при построении логистической регрессии, определим самые важные признаки.

In [30]:
get_cv_metrics(clf, lf_rfe.transform(X_train), y_train, scoring)

fit_time          0.005198
score_time        0.038401
test_accuracy     0.764192
test_precision    0.418278
test_recall       0.324082
test_f1           0.282654
Name: KNeighborsClassifier, dtype: float64

In [31]:
lf_rfe.transform(X_train).shape

(2666, 8)

#### Рекурсивная оценка на основе модели случайного леса

In [32]:
rf_rfe = RFE(RandomForestClassifier(), n_features_to_select=10).fit(X_train, y_train)

Попробуем рекурсивно перебрать все признаки и на основании того, насколько признаки снижают энтропию в модели случайного леса, определим самые важные признаки.

In [33]:
get_cv_metrics(clf, rf_rfe.transform(X_train), y_train, scoring)

fit_time          0.006401
score_time        0.025198
test_accuracy     0.867595
test_precision    0.629624
test_recall       0.279652
test_f1           0.385417
Name: KNeighborsClassifier, dtype: float64

In [34]:
rf_rfe.transform(X_train).shape

(2666, 10)

### Выбор перебором

In [35]:
#sfs = SequentialFeatureSelector(clf).fit(X_train, y_train)

In [36]:
#get_cv_metrics(clf, sfs.transform(X_train), y_train, scoring)

In [37]:
#sfs.transform(X_train).shape

### Выводы по выбору признаков

Минимальное количество признаков с наибольшей эффективностью удалось достичь при использовании одномерного отбора по статистике ХИ-квадрат.

In [38]:
X.columns[chi2_skb.get_support()].values

array(['Number vmail messages', 'Total day minutes', 'Total day charge',
       'Total eve minutes', 'Customer service calls',
       'International plan_Yes'], dtype=object)

В результате выбора признаков из 20 признаков датасета мы получили всего 6, причем качество модели улучшилось в 1.3 раза.

## Итоговая модель

In [39]:
clf.fit(chi2_skb.transform(X_train), y_train)
y_pred = clf.predict(chi2_skb.transform(X_test))

In [40]:
accuracy_score(y_test, y_pred)

0.9115442278860569

In [41]:
precision_score(y_test, y_pred, pos_label=True)

0.7647058823529411

In [42]:
recall_score(y_test, y_pred, pos_label=True)

0.45348837209302323

In [43]:
f1_score(y_test, y_pred, pos_label=True)

0.5693430656934306